In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/zomato_cleaned.csv')
df

,url,online_order,book_table,rate,votes,rest_type,approx_cost(for two people),listed_in(type),listed_in(city),Afghan,...,Spanish,Steak,Street Food,Sushi,Tamil,Tea,Thai,Tibetan,Turkish,Vietnamese
0,https://www.zomato.com/bangalore/jalsa-banasha...,1,1,4.1,775,Casual Dining,800,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
1,https://www.zomato.com/bangalore/spice-elephan...,1,0,4.1,787,Casual Dining,800,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
2,https://www.zomato.com/SanchurroBangalore?cont...,1,0,3.8,918,"Cafe, Casual Dining",800,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,0,0,3.7,88,Quick Bites,300,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
4,https://www.zomato.com/bangalore/grand-village...,0,0,3.8,166,Casual Dining,600,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13222,https://www.zomato.com/bangalore/fujian-expres...,1,0,3.8,182,Food Court,600,Dine-out,Whitefield,0,...,0,0,0,0,0,0,1,0,0,0
13223,https://www.zomato.com/bangalore/nook-aloft-be...,0,1,4.2,411,Casual Dining,1800,Dine-out,Whitefield,0,...,0,0,0,0,0,0,0,0,0,0
13224,https://www.zomato.com/bangalore/seeya-restaur...,0,0,3.3,4,Quick Bites,350,Dine-out,Whitefield,0,...,0,0,0,0,0,0,0,0,0,0
13225,https://www.zomato.com/bangalore/andhra-spices...,0,0,3.2,4,Quick Bites,300,Dine-out,Whitefield,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
x = df.drop(['rate', 'url'], axis=1)
y = df['rate']

In [4]:
cat = x.select_dtypes(include='object').columns
numeric = x.select_dtypes(exclude='object').columns

In [5]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first')
cat_data = ohe.fit_transform(x[cat]).toarray() 

In [6]:
x = pd.concat([x[numeric], pd.DataFrame(cat_data, columns=ohe.get_feature_names(cat))], axis=1)

c:\Users\Yousef\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x , y , test_size=0.2, random_state= 42)

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, classification_report, plot_confusion_matrix

In [10]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train_scaled,y_train)
y_pred_lr = lr.predict(x_test_scaled)
print('R2 for training',lr.score(x_train_scaled,y_train))
print('R2 for testing',lr.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, lr.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_pred_lr))
print('MSE for training',mean_squared_error(y_train, lr.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_pred_lr))
dt_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_pred_lr, 1)})
dt_dataframe

R2 for training 0.36151204319100505
R2 for testing -3.8657692957843907e+24
MAPE for training 0.07934006176694532
MAPE for testing 5022347866.105246
MSE for training 0.12329484109309126
MSE for testing 7.310440922973426e+23


,Actual,Predicted
4111,3.7,3.5
13002,3.6,3.7
4639,3.4,3.8
7377,3.7,3.6
10475,3.6,3.5
...,...,...
11629,4.0,3.7
1056,4.0,4.1
1146,3.3,3.2
4102,3.2,3.9


In [11]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(x_train_scaled,y_train)
y_pred_dtr = dt.predict(x_test_scaled)
print('R2 for training',dt.score(x_train_scaled,y_train))
print('R2 for testing',dt.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, dt.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_pred_dtr))
print('MSE for training',mean_squared_error(y_train, dt.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_pred_dtr))
dt_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_pred_dtr, 1)})
dt_dataframe

R2 for training 0.9995081336047913
R2 for testing 0.283900170934552
MAPE for training 9.016715837748657e-05
MAPE for testing 0.06561800575496592
MSE for training 9.498157074000568e-05
MSE for testing 0.13541950113378684


,Actual,Predicted
4111,3.7,3.6
13002,3.6,3.5
4639,3.4,3.7
7377,3.7,3.7
10475,3.6,3.3
...,...,...
11629,4.0,4.0
1056,4.0,4.3
1146,3.3,3.3
4102,3.2,3.2


In [17]:
dt_dataframe[dt_dataframe['Actual'] != dt_dataframe['Predicted'] ]

,Actual,Predicted
4111,3.7,3.6
13002,3.6,3.5
4639,3.4,3.7
10475,3.6,3.3
10691,4.1,3.9
...,...,...
8898,3.4,3.6
3063,3.4,3.3
7528,3.3,3.9
1056,4.0,4.3


In [18]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(x_train_scaled,y_train)
y_pred_rf = rf.predict(x_test_scaled)
print('R2 for training',rf.score(x_train_scaled,y_train))
print('R2 for testing',rf.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, rf.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_pred_rf))
print('MSE for training',mean_squared_error(y_train, rf.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_pred_rf))
rf_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_pred_rf, 1)})
rf_dataframe

R2 for training 0.9416124796690636
R2 for testing 0.6256560916421345
MAPE for training 0.021142231391416588
MAPE for testing 0.054696061723885116
MSE for training 0.011274887747297012
MSE for testing 0.07079105910198595


,Actual,Predicted
4111,3.7,3.8
13002,3.6,3.5
4639,3.4,3.4
7377,3.7,3.6
10475,3.6,3.3
...,...,...
11629,4.0,4.0
1056,4.0,4.2
1146,3.3,3.3
4102,3.2,3.6


In [19]:
from sklearn.svm import SVR
svm = SVR()
svm.fit(x_train_scaled,y_train)
y_predsvmf = svm.predict(x_test_scaled)
print('R2 for training',svm.score(x_train_scaled,y_train))
print('R2 for testing',svm.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, svm.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_predsvmf))
print('MSE for training',mean_squared_error(y_train, svm.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_predsvmf))
svm_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_predsvmf, 1)})
svm_dataframe


R2 for training 0.577871630758396
R2 for testing 0.38256363802689664
MAPE for training 0.06054419113102935
MAPE for testing 0.07625783388217196
MSE for training 0.08151485028260144
MSE for testing 0.1167615473800317


,Actual,Predicted
4111,3.7,3.5
13002,3.6,3.7
4639,3.4,3.7
7377,3.7,3.6
10475,3.6,3.5
...,...,...
11629,4.0,3.8
1056,4.0,4.0
1146,3.3,3.2
4102,3.2,3.9


In [21]:
svm_dataframe[svm_dataframe['Actual'] != svm_dataframe['Predicted']]

,Actual,Predicted
4111,3.7,3.5
13002,3.6,3.7
4639,3.4,3.7
7377,3.7,3.6
10475,3.6,3.5
...,...,...
3366,4.1,4.0
11629,4.0,3.8
1146,3.3,3.2
4102,3.2,3.9


In [20]:
from  xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(x_train_scaled,y_train)
y_pred_xgb = xgb.predict(x_test_scaled)
print('R2 for training',svm.score(x_train_scaled,y_train))
print('R2 for testing',svm.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, xgb.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_pred_xgb))
print('MSE for training',mean_squared_error(y_train, xgb.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_pred_xgb))

xgb_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_pred_xgb, 1)})
xgb_dataframe



R2 for training 0.577871630758396
R2 for testing 0.38256363802689664
MAPE for training 0.051408168686379235
MAPE for testing 0.06417411900258264
MSE for training 0.058292049914696886
MSE for testing 0.0898291693740876


,Actual,Predicted
4111,3.7,3.8
13002,3.6,3.6
4639,3.4,3.5
7377,3.7,3.6
10475,3.6,3.4
...,...,...
11629,4.0,3.9
1056,4.0,4.2
1146,3.3,3.2
4102,3.2,3.9


In [22]:
y_test_calssified = y_test.apply(lambda x: 1 if x > 3.75 else 0)
y_pred_calssified = pd.Series(y_pred_rf).apply(lambda x: 1 if x > 3.75 else 0)

In [23]:
print(classification_report(y_test_calssified, y_pred_calssified))

              precision    recall  f1-score   support

           0       0.83      0.94      0.88      1511
           1       0.91      0.74      0.81      1135

    accuracy                           0.86      2646
   macro avg       0.87      0.84      0.85      2646
weighted avg       0.86      0.86      0.85      2646



In [24]:
y_train_calssified = y_train.apply(lambda x: 1 if x > 3.75 else 0)
y_pred_train_calssified = pd.Series(rf.predict(x_train_scaled)).apply(lambda x: 1 if x > 3.75 else 0)

In [25]:
print(classification_report(y_train_calssified, y_pred_train_calssified))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      5841
           1       0.98      0.90      0.94      4740

    accuracy                           0.95     10581
   macro avg       0.95      0.94      0.94     10581
weighted avg       0.95      0.95      0.95     10581

